# 模型构造

## 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0830, -0.1644,  0.0392, -0.0114,  0.1201, -0.0032, -0.0586,  0.0094,
          0.0124, -0.1984],
        [-0.2127, -0.0921,  0.0397,  0.0108,  0.0874,  0.0397, -0.0961,  0.0692,
         -0.0745, -0.2173]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

**自定义块**

In [2]:
class MLP(nn.Module):
    """是nn.Module的一个子类，有两个比较重要的函数 __init__ 和 forward"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    # 前向函数
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.0414,  0.0073, -0.0276,  0.1697, -0.2030,  0.0865,  0.0963,  0.1326,
          0.2104, -0.0576],
        [ 0.0620,  0.0524,  0.1148,  0.0090, -0.2636,  0.0728,  0.0934,  0.0344,
          0.1724, -0.1285]], grad_fn=<AddmmBackward0>)

**顺序块**

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block # 特殊容器，ordering dictionary
            
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0043,  0.1946,  0.0698,  0.1076,  0.1260,  0.0270,  0.1481,  0.0621,
         -0.1136, -0.0071],
        [-0.0399,  0.2789,  0.0050,  0.2374,  0.0673,  0.0332,  0.0711,  0.0109,
         -0.2513,  0.0391]], grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)
    

tensor(0.0789, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.3078, grad_fn=<SumBackward0>)